# Risk Liquidity Report

In [1]:
import pandas as pd
from numpy import NaN

In [2]:
# Specify the date and portfolio here
date = '20200630'
portfolio = '19437'
mode='production'
reportingCurrency = 'USD'

## Load Portfolio

1. Load portfolio;
2. Add security id.

In [3]:
from risk_report.data import getIdnType, getPortfolioPositions
from utils.dframe import dictListToDataFrame

df = dictListToDataFrame(getPortfolioPositions(portfolio, date, mode))
df['SecurityId'] =  df.apply(lambda p: getIdnType(p)[0], axis=1)
df[['SecurityId', 'PeriodEndDate', 'LongShortDescription', 'SortKey', 'Quantity']].head()

,SecurityId,PeriodEndDate,LongShortDescription,SortKey,Quantity
0,XS2109438205,2020-06-30,Investments Long,Corporate Bond,1500000.0
1,XS2194361494,2020-06-30,Investments Long,Corporate Bond,9300000.0
2,XS1959497782,2020-06-30,Investments Long,Corporate Bond,3000000.0
3,XS2071413483,2020-06-30,Investments Long,Corporate Bond,2500000.0
4,US00131MAJ27,2020-06-30,Investments Long,Corporate Bond,2000000.0


## Load Liquidity Data

1. Load liquidity data;
2. Show positions with invalid liquidity response.
3. Prepare the special case liquidity file (for bond).

In [4]:
from risk_report.data import getLqaData

lqaData = dictListToDataFrame(getLqaData(date, mode, '|').values())
lqaData = lqaData.drop(columns=[''])
lqaData = lqaData.replace({'N.A.': NaN})
lqaData[lqaData['LQA_LIQUIDATION_HORIZON'].isnull()]

,index,SECURITIES,ERROR CODE,NUM FLDS,LQA_POSITION_TAG_1,LQA_MARKET_PRICE_UNC_PRICE,LQA_TGT_LIQUIDATION_VOLUME,LQA_LIQUIDITY_SECTOR,LQA_TGT_LIQUIDATION_HORIZON,LQA_LIQUIDATION_COST,LQA_TOTAL_LIQUIDATION_COST,LQA_TGT_LIQUIDATION_COST,LQA_LIQUIDATION_HORIZON,LQA_TIME_TO_CASH
1,0,XS2194361494,0.0,10.0,masterlist,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
52,0,XS2188788140,0.0,10.0,masterlist,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
98,0,XS2191371769,0.0,10.0,masterlist,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
103,0,XS2191308563,0.0,10.0,masterlist,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
104,0,9999 HK Equity,0.0,10.0,masterlist,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
149,0,US874060AZ95,0.0,10.0,masterlist,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


## Get Liquidity Category

1. Merge liquidity data with positions;
2. Get liquidity category for those with valid liquidity response;
3. Get liquidity category for certain types of positions (cash, quantity = 0);
4. Get liquidity category with special case file (for bond);
5. Get liquidity category with final override.

In [5]:
dfwLiquidity = df.merge( lqaData
                       , left_on='SecurityId'
                       , right_on='SECURITIES'
                       , how='left')
dfwLiquidity[['SecurityId', 'PeriodEndDate', 'LongShortDescription', 'SortKey', 'Quantity', 'LQA_LIQUIDATION_HORIZON']].head()

,SecurityId,PeriodEndDate,LongShortDescription,SortKey,Quantity,LQA_LIQUIDATION_HORIZON
0,XS2109438205,2020-06-30,Investments Long,Corporate Bond,1500000.0,0.4
1,XS2194361494,2020-06-30,Investments Long,Corporate Bond,9300000.0,NaN
2,XS1959497782,2020-06-30,Investments Long,Corporate Bond,3000000.0,0.6
3,XS2071413483,2020-06-30,Investments Long,Corporate Bond,2500000.0,0.7
4,US00131MAJ27,2020-06-30,Investments Long,Corporate Bond,2000000.0,0.5


### Get Liquidity for (2) and (3)

In [6]:
from risk_report.data import isCash, getQuantity

toLiquiditCategory = lambda daysToCash: \
    'L0' if daysToCash <= 3 else \
    'L1' if daysToCash <= 7 else \
    'L2' if daysToCash <= 10 else 'L3'


isLiquidAsset = lambda position: \
    True if isCash(position) or getQuantity(position) == 0 else False


# Get liquidity from LQA data
dfwLiquidity['Liquidity'] = dfwLiquidity[dfwLiquidity['LQA_LIQUIDATION_HORIZON'].notnull()]['LQA_LIQUIDATION_HORIZON'].apply(toLiquiditCategory)

# Get liquidity for cash and positions not settled yet.
dfwLiquidity.loc[dfwLiquidity.apply(isLiquidAsset, axis=1), 'Liquidity'] = 'L0'

# Show those still not having liquidity data yet
dfwLiquidity[dfwLiquidity['Liquidity'].isnull()][['SecurityId', 'PeriodEndDate', 'SortKey', 'Quantity', 'LQA_LIQUIDATION_HORIZON']]

,SecurityId,PeriodEndDate,SortKey,Quantity,LQA_LIQUIDATION_HORIZON
1,XS2194361494,2020-06-30,Corporate Bond,9300000.0,NaN
55,XS2188788140,2020-06-30,Corporate Bond,10000000.0,NaN
112,XS2191371769,2020-06-30,Corporate Bond,4000000.0,NaN
117,XS2191308563,2020-06-30,Corporate Bond,2500000.0,NaN
118,9999 HK Equity,2020-06-30,Common Stock,79600.0,NaN
163,US874060AZ95,2020-06-30,Corporate Bond,11000000.0,NaN


### Get Liquidity With Special Case Data

In [7]:
from risk_report.data import getLiquiditySpecialCaseData, getBlpData
from risk_report.main_liquidity import getLiquidityCategorySpecialCaseBond
from functools import partial

specialCaseData = dictListToDataFrame(getLiquiditySpecialCaseData(date, mode).values())
dfwSepcialCase = dfwLiquidity.merge( specialCaseData
                                   , left_on='SecurityId'
                                   , right_on='ID'
                                   , how='left')

dfwSepcialCase.loc[dfwSepcialCase['AMT_OUTSTANDING'].notnull(), 'Liquidity'] = \
    dfwSepcialCase[dfwSepcialCase['AMT_OUTSTANDING'].notnull()].apply( 
        partial(getLiquidityCategorySpecialCaseBond, date, getBlpData(date, mode))
      , axis=1
    )


# now let's check how many still did not get liquidity
dfwSepcialCase[dfwSepcialCase['Liquidity'].isnull()][['SecurityId', 'PeriodEndDate', 'Description', 'SortKey', 'Quantity']]

,SecurityId,PeriodEndDate,Description,SortKey,Quantity
118,9999 HK Equity,2020-06-30,NETEASE INC,Common Stock,79600.0


### The Final Override

In [8]:
from risk_report.data import getLiquidityOverride

dfwSepcialCase.loc[dfwSepcialCase['Liquidity'].isnull(), 'Liquidity'] = \
    dfwSepcialCase[dfwSepcialCase['Liquidity'].isnull()]['SecurityId'].apply(lambda sid: getLiquidityOverride()[(date, sid)])

# check again
dfwSepcialCase[dfwSepcialCase['Liquidity'].isnull()][['SecurityId', 'PeriodEndDate', 'Description', 'SortKey', 'Quantity']]

,SecurityId,PeriodEndDate,Description,SortKey,Quantity


### Get Final Outcome

In [9]:
from risk_report.main import marketValueWithFX, getFX

dfwSepcialCase['MarketValue'] = dfwSepcialCase.apply(partial(marketValueWithFX, getFX(date, reportingCurrency)), axis=1)
dfwSepcialCase.groupby('Liquidity')['MarketValue'].sum()

Liquidity
L0    5.129671e+08
L1    7.281579e+07
L2    2.903009e+07
L3    9.676542e+05
Name: MarketValue, dtype: float64

#### Write Output Csv

In [11]:
dfwSepcialCase.groupby('Liquidity')['MarketValue'].sum().to_csv('liquidity_' + date + '.csv')

## Others

Say we want to filter out security list in different liquidity category.

In [12]:
dfwSepcialCase[dfwSepcialCase['Liquidity']=='L1']['SecurityId'].to_csv('liquidity_medium_' + date + '.csv')

## To Do

Next step would be to combine the liquidity report with asset allocation report. This will be done through a merge operation. To do that, we need to:

1. Make sure there is only one row per security id so that we can use this as a key to join two tables. This means positions under the same security needs to be combined.

2. Bloomger and Geneva positions needs to be combined as well. This means we may need to form a new dataframe after loading the positions?